In [57]:
using LinearAlgebra
using cMPO
using OMEinsum
using DelimitedFiles

In [58]:
β = 10.0; D = 8
xxz2D = XXZmodel_2D_helical(1.0)
xxz = XXZmodel(1.0)
ising = TFIsing(1.0,1.0)
pyFolder = "/home/sliang/JuliaCode/mycMPO/pydata"

"/home/sliang/JuliaCode/mycMPO/pydata"

In [59]:
# def tw xxz model
function XXZmodel_tw(Δ::Real)
    if Δ == 0
        return XYmodel()
    else
        sp = pauli(:+); sm = pauli(:-); sz = 0.5 * pauli(:z)
        L = zeros(2, 2, 3)
        L[:,:,1] = 1/√2 * sp ; L[:,:,2] = 1/√2 * sm; L[:,:,3] = √Δ * sz
        R = zeros(2, 2, 3)
        R[:,:,1] = 1/√2 * sm ; R[:,:,2] = 1/√2 * sp; R[:,:,3] = -√Δ * sz
        Q = zeros(2, 2)
        P = zeros(2, 2, 3, 3)
        Tmatrix = CMPO(Q,R,L,P)
        Ut = [0 1 0; 1 0 0; 0 0 -1] # may not be correct
        return PhysModel(Tmatrix, 2, 3, Ut)
    end
end

XXZmodel_tw (generic function with 1 method)

In [60]:
twψ = readCMPS("$(pyFolder)/tw.hdf5", python = true)
twf_py = -0.4465157999549579
xxz_D8 = -0.4465085917120085
twf = free_energy(twψ, xxz.Tmatrix, β)
println("twf - twf_py = ", twf - twf_py) 
println("xxz_D8 - twf_py = ", xxz_D8 - twf_py)  #检查精度问题, 检查 norm(<ψ|ψ>)

twf - twf_py = 1.2227938361308865e-6
xxz_D8 - twf_py = 7.208242949363619e-6


In [61]:
xxz_tw = XXZmodel_tw(1.0)
# 精度问题： Tmatrix 不同
twf_t = free_energy(twψ, xxz_tw.Tmatrix, β)
println("twf_t - twf_py = ", twf_t - twf_py)

twf_t - twf_py = 1.3437384361880689e-6


In [62]:
tw_py_gong_norm = 21.398861689266973
tw_jl_gong_norm = norm(twψ*twψ)
println("|tw_py_gong_norm - tw_jl_gong_norm| = ", abs(tw_py_gong_norm - tw_jl_gong_norm))

|tw_py_gong_norm - tw_jl_gong_norm| = 0.0


In [63]:
tw_py_wang_norm = 30.8915383554522
tw_jl_wang_norm = norm(twψ* xxz_tw.Tmatrix * twψ)
println("|tw_py_wang_norm - tw_jl_wang_norm| = ", abs(tw_py_wang_norm - tw_jl_wang_norm))

|tw_py_wang_norm - tw_jl_wang_norm| = 3.552713678800501e-15


In [64]:
tw_py_gong_lnovlp = 2.765812266575948
tw_jl_gong_lnovlp = log_overlap(twψ, twψ, β)
println("|tw_py_gong_lnovlp - tw_jl_gong_lnovlp| = ", abs(tw_py_gong_lnovlp - tw_jl_gong_lnovlp))

|tw_py_gong_lnovlp - tw_jl_gong_lnovlp| = 2.886579864025407e-14


In [65]:
tw_py_wang_lnovlp = 7.230956828741138
tw_jl_wang_lnovlp = log_overlap(twψ, xxz_tw.Tmatrix * twψ, β)
println("|tw_py_wang_lnovlp - tw_jl_wang_lnovlp| = ", abs(tw_py_wang_lnovlp - tw_jl_wang_lnovlp))

|tw_py_wang_lnovlp - tw_jl_wang_lnovlp| = 1.7763568394002505e-15


In [66]:
fpy = -(tw_py_wang_lnovlp  - tw_py_gong_lnovlp)/β
fjl = -(tw_jl_wang_lnovlp - tw_jl_gong_lnovlp)/β
println("|fpy - fjl| = ", abs(fpy - fjl))
println("|fpy - twf_py| = ", abs(fpy - twf_py))  

|fpy - fjl| = 2.7200464103316335e-15
|fpy - twf_py| = 1.3437384389081153e-6


In [68]:
println("|twf - fpy| = ", abs(twf - fpy))
println("|twf_t - fpy| = ", abs(twf_t - fpy))

|twf - fpy| = 1.2094460277722874e-7
|twf_t - fpy| = 2.7200464103316335e-15


In [ ]:
#ls_p100.hdf5
Ut = xxz2D.Ut
ψr = readCMPS("$(pyFolder)/ls_p100.hdf5", python = true)
ψl = Ut * ψr
lsf1 = free_energy(ψl, ψr, xxz2D.Tmatrix, β)
lsf2 = free_energy(ψr, xxz2D.Tmatrix, β)
lsf_py = -0.446514577807

println("lsf - lsf_py = ", lsf1 - lsf_py) 
println("lsf - xxz_D8 = ", lsf1 - xxz_D8) 

In [ ]:
log_overlap(ψr, ψr, β) ≈ log_overlap(ψl, ψr, β) 

In [ ]:
ψr.R ≈ ψl.R

In [ ]:
# calculate G(τ)
Ntau = 1601
o = make_operator(pauli(:z), 10)
tau = [i for i in range(0,β,length=Ntau)]
Gt = [correlation_2time(τ,o,o',ψl,ψr,xxz2D.Tmatrix, β) for τ in tau];

In [ ]:
Gtau_ResultFile = "$(pyFolder)/gtau_pz_ls_p100.txt"
open(Gtau_ResultFile,"w+") do file
    for i=1: Ntau
        writedlm(file,[tau[i] Gt[i]])
    end
end

In [ ]:
# calculate G(τ) - tw
Gt_tw = [correlation_2time(τ,o,o',twψ, xxz.Tmatrix, β) for τ in tau];

In [ ]:
Gtau_ResultFile_tw= "$(pyFolder)/gtau_pz_tw.txt"
open(Gtau_ResultFile_tw,"w+") do file
    for i=1: Ntau
        writedlm(file,[tau[i] Gt_tw[i]])
    end
end

In [ ]:
# Check free energy function 
IsingData = load("/data/sliang/CMPO/ising/D_8/ug_1.0.jld")
ψi = tocmps(IsingData[string(β)][2])
free_energy(ψi, ising.Tmatrix, β) ≈ -1.2745494893094984

In [ ]:
#test hdf5
#a2D = [1 2; 3 4]
# numpy a3D[0,:,:] = [1 2; 3 4] ; a3D[1,:,:] = [5 6; 7 8]
test = h5open("test.hdf5", "r")
a2D = read(test["a2D"]) |> transpose
a3D = read(test["a3D"])
a3D = ein"ijk->jik"(a3D)

b2D = [1 2; 3 4]
b3D = [1; 3;; 2; 4;;; 5; 7;; 6; 8] ; #b3D[:,:,1] = [1 2; 3 4] ; b3D[:,:,2] = [5 6; 7 8] ;

println("a2D == b2D ? ", a2D == b2D)
println("a3D == b3D ? ", a3D == b3D)